In [3]:
from skimage.morphology import erosion, dilation, opening, closing
from skimage.morphology import disk
from skimage import data, io
from matplotlib import pyplot as plt
import numpy as np
from skimage.transform import rescale, resize
from skimage import img_as_ubyte, img_as_float
from skimage import data, io, filters
from matplotlib.pyplot import imshow, show, subplot, title, get_cmap
from skimage.segmentation import mark_boundaries, find_boundaries
from skimage.filters import threshold_otsu
from skimage.util import invert
import cv2
from IPython.display import Image
import pytesseract
import imutils
import easyocr
from easyocr import Reader
import torch
import glob


In [4]:
for fn in glob.glob('*.jpg'):
    img = cv2.imread(fn)
    cv2.imshow("Orignal Image", img)
    cv2.waitKey()
    cv2.destroyAllWindows()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    Filter = cv2.bilateralFilter(gray, 11, 17, 17)
    edges = cv2.Canny(Filter, 20, 200)
    kernel = np.ones((1, 1), np.uint8)
    cnt = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(cnt)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]
    l = None
    for c in contours:
        arc = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.06*arc, True)
        if len(approx) == 4:
            l = approx
            break
    mask = np.zeros(gray.shape, np.uint8)
    res = cv2.drawContours(mask, [l], 0, 255, -1)
    res = cv2.bitwise_and(img, img, mask=mask)
    (x, y) = np.where(mask == 255)
    (xt, yt) = (np.min(x), np.min(y))
    (xr, yr) = (np.max(x), np.max(y))
    crp = res[xt:xr+1, yt:yr+1]
    cv2.imshow("Cropped Image", crp)
    cv2.waitKey()
    cv2.destroyAllWindows()
    contours2, hierarchy2 = cv2.findContours(
        mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    Roi = None
    for j in contours2:
        area = cv2.contourArea(j)
        if (area > 1000):
            x, y, w, h = cv2.boundingRect(j)
            Roi = cv2.rectangle(img, (x, y), (x + w, y + h), (255, 36, 12), 5)
            img[y:y+h, x:x+w]
    reader = easyocr.Reader(['en'])
    #for arabic image
    # reader = easyocr.Reader(['ar'])
    read = reader.readtext(crp)
    text = read[0][-2]

    
    print("--")
    print("Car Plate Number is :")
    print(text)
    print("--")
    font = cv2.FONT_HERSHEY_SIMPLEX
    resu = cv2.putText(img, text=text, org=(approx[0][0][0]-100, approx[1][0][1]-50),
                   fontFace=font, fontScale=1, color=(0, 255,255), thickness=6, lineType=cv2.LINE_AA)
    cv2.imshow("Output",resu)
    cv2.waitKey()
    cv2.destroyAllWindows()
  

 

--
Car Plate Number is :
H982 FKL
--
--
Car Plate Number is :
MH12 DE1433
--
--
Car Plate Number is :
GD7O Eoo
--
--
Car Plate Number is :
LR33 TEE
--
--
Car Plate Number is :
HR 26 BR 9044
--
--
Car Plate Number is :
HR 26 DA 7330
--
